# Programacion lineal con PuLP

In [1]:
import pulp as p

## Problemas simples

Maximize profit:

A company produces two different products of wood: chairs and tables.

- The profit per chair is $20 and $30 per table.
- A chair requires 1 hr of finishing labor and 2 for carpentry
- A table requires 2 hrs of finishing labor and 1 for carpentry
- The available time for finishing and carpentry labors are
100 hrs each.


In [2]:
prob = p.LpProblem("Maximize_profit", p.LpMaximize)

# Variables
A = p.LpVariable("A", lowBound=0, cat="Integer")
B = p.LpVariable("B", lowBound=0, cat="Integer")

# Funcion objetivo
prob += A * 20 + B * 30, "Profit"

# Restricciones
prob += A + B * 2 <= 100, "time_limit1"
prob += A * 2 + B <= 100, "time_limit2"

prob.solve()
print(p.LpStatus[prob.status])

print(f"Production of chairs: {A.varValue}")
print(f"Production of chairs: {B.varValue}")

# Print objective function value

print(p.value(prob.objective))

Optimal
Production of chairs: 32.0
Production of chairs: 34.0
1660.0


## Planificacion de produccion

In [3]:
# List (Time Periods)
t = [0, 1, 2, 3, 4, 5, 6]

# DATA
demand = {1:100, 2:100, 3:150, 4:200, 5:150, 6:100} # Demand data
UPC = {1:7, 2:8, 3:8, 4:8, 5:7, 6:8} # Unit Production Cost (Excluding Labor)
UHC = {1:3, 2:4, 3:4, 4:4, 5:3, 6:2} # Unit Holding Cost
URLC = {1:15, 2:15, 3:18, 4:18, 5:15, 6:15} # Unit Regular Labor Cost
UOLC = {1:22.5, 2:22.5, 3:27, 4:27, 5:22.5, 6:22.5} # Unit Overtime Labor Cost
R_MH = {1:120, 2:130, 3:120, 4:150, 5:100, 6:100} # Available Man-hours R (Regular time) Labor
O_MH = {1:30, 2:40, 3:40, 4:30, 5:30, 6:30} # Available Man-hours O (Overtime) Labor

# Setting problem
prob = p.LpProblem("Aggregate_Production", p.LpMinimize)

# Desicion Variables
Xt = p.LpVariable.dicts("Quantity_Produced", t, 0)
It = p.LpVariable.dicts("Inventory", t, 0)
Rt = p.LpVariable.dicts("R_Labor_Used", t, 0)
Ot = p.LpVariable.dicts("O_Labor_Used", t, 0)

# Objective Function
prob += p.lpSum(UPC[i]*Xt[i] for i in t[1:])+p.lpSum(UHC[i]*It[i] for i in t[1:])+p.lpSum(URLC[i]*Rt[i] for i
in t[1:])+p.lpSum(UOLC[i]*Ot[i] for i in t[1:])


# Restricciones
initial_inventory = 3
prob += It[0] == initial_inventory

for i in t[1:]:
    prob += (Xt[i] + It[i-1] - It[i]) == demand[i]
for i in t[1:]:
    prob += Xt[i] - Rt[i] - Ot[i] == 0
for i in t[1:]:
    prob += Rt[i] <= R_MH[i]
for i in t[1:]:
    prob += Ot[i] <= O_MH[i] 
    
prob.solve()
print("Solution Status = ", p.LpStatus[prob.status])
# Print the solution of the Decision Variables
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Solution Status =  Optimal
Inventory_0 = 3.0
Inventory_1 = 23.0
Inventory_2 = 70.0
Inventory_3 = 40.0
Inventory_4 = 20.0
O_Labor_Used_2 = 17.0
O_Labor_Used_4 = 30.0
O_Labor_Used_5 = 30.0
Quantity_Produced_1 = 120.0
Quantity_Produced_2 = 147.0
Quantity_Produced_3 = 120.0
Quantity_Produced_4 = 180.0
Quantity_Produced_5 = 130.0
Quantity_Produced_6 = 100.0
R_Labor_Used_1 = 120.0
R_Labor_Used_2 = 130.0
R_Labor_Used_3 = 120.0
R_Labor_Used_4 = 150.0
R_Labor_Used_5 = 100.0
R_Labor_Used_6 = 100.0


In [4]:
import pandas as pd

In [5]:
demand = pd.read_csv("monthly_demand.csv")
factories = pd.read_csv("factory_variables.csv")

In [6]:
prob = p.LpProblem("Minimizacion_costos", p.LpMinimize)

months = list(range(1,12))

x1 = p.LpVariable.dicts("Fabrica_1", months, lowBound=0)
x2 = p.LpVariable.dicts("Fabrica_2", months, lowBound=0)

Fs_A = p.LpVariable("Estado_Fabrica_A", months, cat="Binary", lowBound=0, upBound=1)
Fs_B = p.LpVariable("Estado_Fabrica_B", months, cat="Binary", lowBound=0, upBound=1)

Vc = p.LpVariable("Variable_Costs", lowBound=0)


# Funcion objetivo
prob += p.lpSum([x1*vc])

TypeError: LpVariable.__init__() got multiple values for argument 'lowBound'